In [1]:
from scipy.stats import skew, kurtosis
from sklearn.svm import SVC

In [3]:
import numpy as np
from context import Context
from data import Data
from copy import deepcopy
from baselines import cross_val, multi_run
from util import Metric

Context.data_dir = "/home/foumani/data/solar_flare_prediction"
Context.files_df_filename = "all_files.csv"
Context.files_np_filename = "full_data_X_1_25.npy"
Context.log_dir = "/home/foumani/workspace/solar_flare_prediction/log"
context = Context(train_n=[400, 400], run_times=3)
data = Data()

Reading all files df ...
Reading all files np ...


In [4]:
def extract_features(data):
    features = np.empty(shape=(len(data), 24, 5))
    features[:, :, 0] = np.median(data, axis=2)
    features[:, :, 1] = np.std(data, axis=2)
    features[:, :, 2] = skew(data, axis=2)
    features[:, :, 3] = kurtosis(data, axis=2)
    features[:, :, 4] = deepcopy(data[:, :, -1])
    features = features.reshape((len(data), 24 * features.shape[2]))
    return features


def preprocess(X, y):
    X = extract_features(X)
    indices = ~np.isnan(X).any(axis=1)
    return X[indices], y[indices]

In [5]:
def baseline_svm(X_train, y_train, X_val, y_val, X_test, y_test):
    X_train, y_train = preprocess(X_train, y_train)
    X_val, y_val = preprocess(X_val, y_val)
    svc = SVC()
    svc.fit(X_train, y_train)
    y_val_pred = svc.predict(X_val)
    return Metric(y_true=y_val, y_pred=y_val_pred, binary=True)

In [ ]:
cross_val(context, data, baseline_svm)

In [6]:
multi_run(context, data, baseline_svm, True)

run no. 1:
Training [2, 3, 4, 5], Val None, Test 1


100%|██████████| 800/800 [00:00<00:00, 19960.64files/s]


UnboundLocalError: local variable 'X_val_np' referenced before assignment